In [1]:
import pandas as pd
import json

In [7]:
df = pd.read_json('https://dataex.ohchr.org/uhri/export-results/export-full-en.json', columns=['Text', 'DocumentId', 'AnnotationType', 'PublicationDate', 'Countries'])

TypeError: read_json() got an unexpected keyword argument 'columns'

In [3]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df', 431233959), ('open', 144), ('pd', 72)]

In [4]:
df = df[df.AnnotationType == '- Concerns/Observations']

In [52]:
# df = df['PublicationDate'] = pd.to_datetime(df['PublicationDate'],  format='%Y-%m-%dT%H:%M:%S')

In [53]:
# df.sort_values(by=['PublicationDate', 'DocumentId'], ascending=False, inplace=True)

In [5]:
df = df.drop(columns=['AnnotationId', 'Symbol',
       'AnnotationType', 'Sdgs',
       'UprPositions', 'UprSession', 'UprRecommendingCountry', 'UprCycle',
       'UprRecommendingRegions', 'Regions', 'PublicationDateOnUhri', 'Body', 'AffectedPersons', 'Themes'])

In [6]:
df.columns

Index(['Text', 'DocumentId', 'PublicationDate', 'Countries'], dtype='object')

In [8]:
with open('/home/me/projects/worldcrisismap/crisis_database/mappers/uhri_country_mapping.json', 'r') as f:
    country_dict = json.load(f)

In [9]:
# loop through each single document
for doc_id in set(df.DocumentId.values):
    entries = df[df.DocumentId == doc_id]

    # these information are needed:
    # - date
    # - country name
    # - geojson paths (from country iso3s)
    # - lat / lon
    # - title
    # - text

    # all unique country codes involved in the document
    country_names_involved = list(set(sum(list(entries.Countries.values), start=[])))

    # if there are no country names involved, skip entry
    if len(country_names_involved) == 0:
        continue
    
    date = entries.PublicationDate.values[0]
    country_name = country_names_involved[0]  # only one first country name is needed
    
    geojson = ['countries/' + country_dict[country]['iso3'] + '.json' for country in country_names_involved]
    
    lat, lon = country_dict[country_name]['lat'], country_dict[country_name]['lon']

    title = 'Human Right Violations in ' + country_name
    text = '<br>'.join(list(entries.Text.values))